In [1]:
from BradenNebularLines import emission
from BradenNebularLines import emission_interpolator

-6.0 1.0 0.5 -1.0 6.0 0.5 3.0 6.0 0.1
(25, 6975)
15 15 31


In [2]:
import os
import sys
import copy

import numpy as np
import yt
from yt.frontends.ramses.field_handlers import RTFieldFileHandler

# import galaxy_visualization

In [5]:
filename = "/Users/lamoreau/python/ASpec/SimulationFiles/output_00273/info_00273.txt"

lines=["H1_6562.80A","O1_1304.86A","O1_6300.30A","O2_3728.80A","O2_3726.10A",
       "O3_1660.81A","O3_1666.15A","O3_4363.21A","O3_4958.91A","O3_5006.84A", 
       "He2_1640.41A","C2_1335.66A","C3_1906.68A","C3_1908.73A","C4_1549.00A",
       "Mg2_2795.53A","Mg2_2802.71A","Ne3_3868.76A","Ne3_3967.47A",
       "N5_1238.82A",
       "N5_1242.80A","N4_1486.50A","N3_1749.67A","S2_6716.44A","S2_6730.82A"]

wavelengths=[6562.80, 1304.86, 6300.30, 3728.80, 3726.10, 1660.81, 1666.15,
             4363.21, 4958.91, 5006.84, 1640.41, 1335.66,
             1906.68, 1908.73, 1549.00, 2795.53, 2802.71, 3868.76,
             3967.47, 1238.82, 1242.80, 1486.50, 1749.67, 6716.44, 6730.82]

cell_fields = [
    "Density",
    "x-velocity",
    "y-velocity",
    "z-velocity",
    "Pressure",
    "Metallicity",
    "xHI",
    "xHII",
    "xHeII",
    "xHeIII",
]

epf = [
    ("particle_family", "b"),
    ("particle_tag", "b"),
    ("particle_birth_epoch", "d"),
    ("particle_metallicity", "d"),
]

# Ionization Parameter Field
# Based on photon densities in bins 2-4
# Don't include bin 1 -> Lyman Werner non-ionizing
def _ion_param(field, data):
    p = RTFieldFileHandler.get_rt_parameters(ds).copy()
    p.update(ds.parameters)

    cgs_c = 2.99792458e10     #light velocity

    # Convert to physical photon number density in cm^-3
    pd_2 = data['ramses-rt','Photon_density_2']*p["unit_pf"]/cgs_c
    pd_3 = data['ramses-rt','Photon_density_3']*p["unit_pf"]/cgs_c
    pd_4 = data['ramses-rt','Photon_density_4']*p["unit_pf"]/cgs_c

    photon = pd_2 + pd_3 + pd_4

    return photon/data['gas', 'number_density']


def _my_temperature(field, data):
    #y(i): abundance per hydrogen atom
    XH_RAMSES=0.76 #defined by RAMSES in cooling_module.f90
    YHE_RAMSES=0.24 #defined by RAMSES in cooling_module.f90
    mH_RAMSES=yt.YTArray(1.6600000e-24,"g") #defined by RAMSES in cooling_module.f90
    kB_RAMSES=yt.YTArray(1.3806200e-16,"erg/K") #defined by RAMSES in cooling_module.f90

    dn=data["ramses","Density"].in_cgs()
    pr=data["ramses","Pressure"].in_cgs()
    yHI=data["ramses","xHI"]
    yHII=data["ramses","xHII"]
    yHe = YHE_RAMSES*0.25/XH_RAMSES
    yHeII=data["ramses","xHeII"]*yHe
    yHeIII=data["ramses","xHeIII"]*yHe
    yH2=1.-yHI-yHII
    yel=yHII+yHeII+2*yHeIII
    mu=(yHI+yHII+2.*yH2 + 4.*yHe) / (yHI+yHII+yH2 + yHe + yel)
    return pr/dn * mu * mH_RAMSES / kB_RAMSES


# TODO see if it works in emission.py
# Luminosity field
# Cloudy Intensity obtained assuming height = 1cm
# Return intensity values erg/s/cm**2
# Multiply intensity at each pixel by volume of pixel -> luminosity
def get_luminosity(line):
   def _luminosity(field, data):
      return data['gas', 'flux_' + line]*data['gas', 'volume']
   return copy.deepcopy(_luminosity)


#number density of hydrogen atoms
def _my_H_nuclei_density(field, data):
    dn=data["ramses","Density"].in_cgs()
    XH_RAMSES=0.76 #defined by RAMSES in cooling_module.f90
    YHE_RAMSES=0.24 #defined by RAMSES in cooling_module.f90
    mH_RAMSES=yt.YTArray(1.6600000e-24,"g") #defined by RAMSES in cooling_module.f90

    return dn*XH_RAMSES/mH_RAMSES


def _pressure(field, data):
    if 'hydro_thermal_pressure' in dir(ds.fields.ramses): # and 
        #'Pressure' not in dir(ds.fields.ramses):
        return data['ramses', 'hydro_thermal_pressure']


def _xHI(field, data):
    if 'hydro_xHI' in dir(ds.fields.ramses): # and \
        #'xHI' not in dir(ds.fields.ramses):
        return data['ramses', 'hydro_xHI']


def _xHII(field, data):
    if 'hydro_xHII' in dir(ds.fields.ramses): # and \
        #'xHII' not in dir(ds.fields.ramses):
        return data['ramses', 'hydro_xHII']


def _xHeII(field, data):
    if 'hydro_xHeII' in dir(ds.fields.ramses): # and \
        #'xHeII' not in dir(ds.fields.ramses):
        return data['ramses', 'hydro_xHeII']


def _xHeIII(field, data):
    if 'hydro_xHeIII' in dir(ds.fields.ramses): # and \
        #'xHeIII' not in dir(ds.fields.ramses):
        return data['ramses', 'hydro_xHeIII']


In [8]:
'''
-------------------------------------------------------------------------------
Load Simulation Data
Add Derived Fields
-------------------------------------------------------------------------------
'''

ds = yt.load(filename, extra_particle_fields=epf)

ds.add_field(
    ("gas","number_density"),
    function=_my_H_nuclei_density,
    sampling_type="cell",
    units="1/cm**3",
    force_override=True
)


ds.add_field(
    ("ramses","Pressure"),
    function=_pressure,
    sampling_type="cell",
    units="1",
    #force_override=True
)

ds.add_field(
    ("ramses","xHI"),
    function=_xHI,
    sampling_type="cell",
    units="1",
    #force_override=True
)

ds.add_field(
    ("ramses","xHII"),
    function=_xHII,
    sampling_type="cell",
    units="1",
    #force_override=True
)

ds.add_field(
    ("ramses","xHeII"),
    function=_xHeII,
    sampling_type="cell",
    units="1",
    #force_override=True
)

ds.add_field(
    ("ramses","xHeIII"),
    function=_xHeIII,
    sampling_type="cell",
    units="1",
    #force_override=True
)

ds.add_field(
    ("gas","my_temperature"),
    function=_my_temperature,
    sampling_type="cell",
    # TODO units
    #units="K",
    #units="K*cm**3/erg",
    units='K*cm*dyn/erg',
    force_override=True
)

# Ionization parameter
ds.add_field(
    ('gas', 'ion_param'),
    function=_ion_param,
    sampling_type="cell",
    units="cm**3",
    force_override=True
)

ds.add_field(
    ("gas","my_H_nuclei_density"),
    function=_my_H_nuclei_density,
    sampling_type="cell",
    units="1/cm**3",
    force_override=True
)

# Normalize by Density Squared Flag
dens_normalized = True
if dens_normalized: 
    units = '1/cm**6'
else:
    units = '1'

# Instance of EmissionLineInterpolator for line list at filename
print(os.getcwd())
line_list = os.path.join(os.getcwd(), 'BradenNebularLines/linelist.dat')
emission_interpolator = emission_interpolator.EmissionLineInterpolator(line_list, lines)

# Add flux and luminosity fields for all lines in the list
for i, line in enumerate(lines):
    ds.add_field(
        ('gas', 'flux_' + line),
        function=emission_interpolator.get_line_emission(
            i, dens_normalized=dens_normalized
        ),
        sampling_type='cell',
        units=units,
        force_override=True
    )
    # TODO change get_line_emission to accept line not idx

    ds.add_field(
        ('gas', 'luminosity_' + line),
        function=emission_interpolator.get_luminosity(lines[i]),
        #function=get_luminosity(lines[i]),
        sampling_type='cell',
        units='1/cm**3',
        force_override=True
    )

ad = ds.all_data()
print(ds.field_list)


yt : [WARNING  ] 2025-12-06 17:36:51,277 `namelist.txt` file found but missing package f90nml to read it:
Traceback (most recent call last):
  File "/Users/lamoreau/merlinproto/lib/python3.10/site-packages/yt/frontends/ramses/data_structures.py", line 1139, in read_namelist
    nml = f90nml.read(f)
  File "/Users/lamoreau/merlinproto/lib/python3.10/site-packages/yt/utilities/on_demand_imports.py", line 39, in __call__
    raise self.error
  File "/Users/lamoreau/merlinproto/lib/python3.10/site-packages/yt/utilities/on_demand_imports.py", line 77, in inner
    return func(self)
  File "/Users/lamoreau/merlinproto/lib/python3.10/site-packages/yt/utilities/on_demand_imports.py", line 420, in read
    from f90nml import read
ModuleNotFoundError: No module named 'f90nml'
Something went wrong while trying to lazy-import f90nml. Please make sure that f90nml is properly installed.
If the problem persists, please file an issue at https://github.com/yt-project/yt/issues/new
yt : [INFO     ] 2025

/Users/lamoreau/python/ASpec/BradenNebularLines/ashiq_spectroscopy_tools
minU=-6.0, maxU=1.0, stepU=0.5, minN=-1.0, maxN=6.0, stepN=0.5, minT=3.0, maxT=6.0, stepT=0.1
Line List Shape = (25, 6975)
15 15 31
[('gravity', 'Potential'), ('gravity', 'x-acceleration'), ('gravity', 'y-acceleration'), ('gravity', 'z-acceleration'), ('io', 'particle_birth_epoch'), ('io', 'particle_family'), ('io', 'particle_identity'), ('io', 'particle_mass'), ('io', 'particle_metallicity'), ('io', 'particle_position_x'), ('io', 'particle_position_y'), ('io', 'particle_position_z'), ('io', 'particle_refinement_level'), ('io', 'particle_tag'), ('io', 'particle_velocity_x'), ('io', 'particle_velocity_y'), ('io', 'particle_velocity_z'), ('nbody', 'particle_mass'), ('nbody', 'particle_position_x'), ('nbody', 'particle_position_y'), ('nbody', 'particle_position_z'), ('nbody', 'particle_velocity_x'), ('nbody', 'particle_velocity_y'), ('nbody', 'particle_velocity_z'), ('ramses', 'Density'), ('ramses', 'Metallicity'), (

In [9]:
ha_flux = ad[("gas", "flux_O1_1304.86A")]
ha_lum  = ad[("gas", "luminosity_O1_1304.86A")]

yt : [INFO     ] 2025-12-06 17:37:31,086 Identified   384/  384 intersecting domains (  385 through hilbert key indexing)


In [10]:
print(ha_flux.value)
print(ha_flux.units)
cell_vol = ad["cell_volume"]
print(cell_vol)

[0. 0. 0. ... 0. 0. 0.]
cm**(-6)
[6.54221705e+69 6.54221705e+69 6.54221705e+69 ... 6.54221705e+69
 6.54221705e+69 6.54221705e+69] cm**3


In [11]:
# --------------------------------------------
# INPUT: list of emission lines (exact names)
# --------------------------------------------
lines = np.array([
    'H1 6562.80A', 'O1 1304.86A', 'O1 6300.30A', 'O2 3728.80A',
    'O2 3726.10A', 'O3 1660.81A', 'O3 1666.15A', 'O3 4363.21A',
    'O3 4958.91A', 'O3 5006.84A', 'He2 1640.41A', 'C2 1335.66A',
    'C3 1906.68A', 'C3 1908.73A', 'C4 1549.00A', 'Mg2 2795.53A',
    'Mg2 2802.71A', 'Ne3 3868.76A', 'Ne3 3967.47A', 'N5 1238.82A',
    'N5 1242.80A', 'N4 1486.50A', 'N3 1749.67A', 'S2 6716.44A',
    'S2 6730.82A'
])
# --------------------------------------------
# Convert "O1 1304.86A" → "O1_1304.86A"
# --------------------------------------------
lines = np.array([l.replace(" ", "_") for l in lines])

# -------------------------------------------------------------------------
# PREP: load all gas data and cell volume once (faster than inside loop)
# -------------------------------------------------------------------------
ad = ds.all_data()
cell_volume = ad["cell_volume"]   # in cm^3
print(cell_volume)
# ----------------------------------------------------
# OUTPUT ARRAYS
# ----------------------------------------------------
total_fluxes = []
total_luminosities = []

yt : [INFO     ] 2025-12-06 17:38:25,830 Identified   162/  384 intersecting domains (  385 through hilbert key indexing)


[6.54221705e+69 6.54221705e+69 6.54221705e+69 ... 6.54221705e+69
 6.54221705e+69 6.54221705e+69] cm**3


In [12]:
# ----------------------------------------------------
# MAIN LOOP — compute integrated flux & luminosity
# ----------------------------------------------------
for line in lines:
    # field names MUST match how you added them
    flux_field = ("gas", f"flux_{line}")
    lum_field  = ("gas", f"luminosity_{line}")

    # read per-cell fields
    flux = ad[flux_field]
    lum  = ad[lum_field]

    # integrate over volume
    total_flux = (flux * cell_volume).sum().value        # remove YT units
    total_lum  = (lum  * cell_volume).sum().value

    total_fluxes.append(total_flux)
    total_luminosities.append(total_lum)

# convert to numpy arrays
total_fluxes = np.array(total_fluxes)
total_luminosities = np.array(total_luminosities)


KeyboardInterrupt: 

In [ ]:
print(total_fluxes)
print(total_luminosities)

[5.18904630e+42 5.13051603e+34 8.39807369e+36 1.44530711e+37
 2.64560523e+37 7.60934844e+35 2.22841703e+36 8.75633317e+35
 8.34803986e+36 2.49071741e+37 1.42540867e+38 4.18735412e+35
 1.37389118e+37 1.16740056e+37 3.98588138e+29 3.53917728e+37
 1.78793091e+37 5.25355092e+35 1.59453244e+35 9.19283542e+32
 4.58610157e+32 9.81514974e+32 1.18496299e+36 8.41942573e+36
 1.28071980e+37]
[2.78858059e+112 1.82011469e+090 3.04909372e+092 8.33011182e+092
 6.06487483e+092 2.19520486e+090 6.42866591e+090 2.57975902e+090
 2.75297490e+091 8.21375469e+091 1.54447040e+104 6.44959173e+090
 6.61707130e+091 4.48001048e+091 7.31939871e+085 8.96403921e+092
 4.53486991e+092 1.55752760e+090 4.72734506e+089 1.69261740e+089
 8.44417755e+088 2.24642905e+088 3.42923548e+090 9.51327019e+092
 6.97538765e+092]


In [ ]:
# saving total flux and lumonisty for each line for the entire snapshot / box
import csv

filen = "line_flux_luminosity_results_whole_simulation_snapshot.csv"

with open(filen, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Line", "Total_Flux", "Total_Luminosity"])

    for line, flux, lum in zip(lines, total_fluxes, total_luminosities):
        writer.writerow([line, flux, lum])

print("Saved:", filen)

Saved: line_flux_luminosity_results_whole_simulation_snapshot.csv


In [13]:
# creating proj of a boxed region
cx = np.mean(ad["star", "particle_position_x"])
cy = np.mean(ad["star", "particle_position_y"])
cz = np.mean(ad["star", "particle_position_z"])
center = [cx, cy, cz]
halfa = ds.quan(20, "kpc")

low_edge = [center[0] - halfa, center[1] - halfa, center[2] - halfa]
high_edge = [center[0] + halfa, center[1] + halfa, center[2] + halfa]

cube_region = ds.region(center, low_edge, high_edge)

ad_box = cube_region


cell_volume_box = ad_box["cell_volume"]   # in cm^3
print(cell_volume)
# ----------------------------------------------------
# OUTPUT ARRAYS
# ----------------------------------------------------
total_fluxes_box = []
total_luminosities_box = []

yt : [INFO     ] 2025-12-06 17:45:20,452 Identified   161/  384 intersecting domains (  384 through hilbert key indexing)


[6.54221705e+69 6.54221705e+69 6.54221705e+69 ... 6.54221705e+69
 6.54221705e+69 6.54221705e+69] cm**3


# Smaller region speedup

In [14]:
# ----------------------------------------------------
# MAIN LOOP — compute integrated flux & luminosity
# ----------------------------------------------------
for line in lines:
    # field names MUST match how you added them
    flux_field = ("gas", f"flux_{line}")
    lum_field  = ("gas", f"luminosity_{line}")

    # read per-cell fields (with units)
    flux = ad_box[flux_field]                                                                                                          
    lum  = ad_box[lum_field]           # units: erg/s

    # integrate over volume
    total_flux = (flux * cell_volume_box).sum()  # units: erg/s
    total_lum  = (lum).sum()                     # already erg/s

    total_fluxes_box.append(total_flux)
    total_luminosities_box.append(total_lum)

# convert to yt arrays or numpy with units if needed
total_fluxes_box = np.array(total_fluxes_box) # unit: erg/s/cm^2
total_luminosities_box = np.array(total_luminosities_box)    #unit: erg/s

In [15]:
print(total_fluxes_box)
print(total_luminosities_box)

[8.83587566e+41 5.13051603e+34 8.39807369e+36 1.44530711e+37
 2.64560523e+37 7.60934844e+35 2.22841703e+36 8.75633317e+35
 8.34803986e+36 2.49071741e+37 1.42517028e+38 4.18735412e+35
 1.37389118e+37 1.16740056e+37 3.98588138e+29 3.53917728e+37
 1.78793091e+37 5.25355092e+35 1.59453244e+35 9.19283542e+32
 4.58610157e+32 9.81514974e+32 1.18496299e+36 8.41942573e+36
 1.28071980e+37]
[8.83587566e+41 5.13051603e+34 8.39807369e+36 1.44530711e+37
 2.64560523e+37 7.60934844e+35 2.22841703e+36 8.75633317e+35
 8.34803986e+36 2.49071741e+37 1.42517028e+38 4.18735412e+35
 1.37389118e+37 1.16740056e+37 3.98588138e+29 3.53917728e+37
 1.78793091e+37 5.25355092e+35 1.59453244e+35 9.19283542e+32
 4.58610157e+32 9.81514974e+32 1.18496299e+36 8.41942573e+36
 1.28071980e+37]


# For loops are dead, long live vecotrization

The new sheriff is in town and he is the fastest draw in the west...

21x speedup... GO

In [19]:
flux_fields = [("gas", f"flux_{line}") for line in lines]
lum_fields  = [("gas", f"luminosity_{line}") for line in lines]

# Load all fields at once (yt handles this efficiently)
flux_data = np.vstack([ad_box[f] for f in flux_fields])  # shape: (Nlines, Ncells) #.to("erg/s/cm**2").d
lum_data  = np.vstack([ad_box[f]        for f in lum_fields])  # shape: (Nlines, Ncells) #.to("erg/s").d

cell_vol  = cell_volume_box.to("cm**3").d[None, :]     # broadcast to shape (1, Ncells)

# Vectorized integration
total_fluxes_box        = (flux_data * cell_vol).sum(axis=1)
total_luminosities_box  = lum_data.sum(axis=1)

In [20]:
print(total_fluxes_box)
print(total_luminosities_box)

[8.83587566e+41 5.13051603e+34 8.39807369e+36 1.44530711e+37
 2.64560523e+37 7.60934844e+35 2.22841703e+36 8.75633317e+35
 8.34803986e+36 2.49071741e+37 1.42517028e+38 4.18735412e+35
 1.37389118e+37 1.16740056e+37 3.98588138e+29 3.53917728e+37
 1.78793091e+37 5.25355092e+35 1.59453244e+35 9.19283542e+32
 4.58610157e+32 9.81514974e+32 1.18496299e+36 8.41942573e+36
 1.28071980e+37] cm**(-6)
[8.83587566e+41 5.13051603e+34 8.39807369e+36 1.44530711e+37
 2.64560523e+37 7.60934844e+35 2.22841703e+36 8.75633317e+35
 8.34803986e+36 2.49071741e+37 1.42517028e+38 4.18735412e+35
 1.37389118e+37 1.16740056e+37 3.98588138e+29 3.53917728e+37
 1.78793091e+37 5.25355092e+35 1.59453244e+35 9.19283542e+32
 4.58610157e+32 9.81514974e+32 1.18496299e+36 8.41942573e+36
 1.28071980e+37] cm**(-3)


In [21]:
# saving total flux and lumonisty for each line for the entire snapshot / box
import csv

filen = "line_flux_luminosity_results_whole_simulation_snapshot.csv"

with open(filen, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Line", "Total_Flux", "Total_Luminosity"])

    for line, flux, lum in zip(lines, total_fluxes, total_luminosities):
        writer.writerow([line, flux, lum])

print("Saved:", filen)

Saved: line_flux_luminosity_results_whole_simulation_snapshot.csv
